In [6]:
import pandas as pd
import datetime
dfs = []

for day in range(1, 31):      # 1 → 30
    filename = f"/Users/nguyentran0703/Downloads/BDC HACKATHON/Bus_route_data/raw_GPS/anonymized_raw_2025-04-{day:02d}.csv"
    df = pd.read_csv(filename)
    dfs.append(df)

raw_all = pd.concat(dfs, ignore_index=True)



/var/folders/nx/tm25jbz54jjdb9mb8c28s0m00000gn/T/ipykernel_25497/1909453941.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
/var/folders/nx/tm25jbz54jjdb9mb8c28s0m00000gn/T/ipykernel_25497/1909453941.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [7]:

raw_all['datetime'] = pd.to_datetime(raw_all['datetime'])
start = pd.to_datetime('05:00').time()
end = pd.to_datetime('22:15').time()

raw_all = raw_all[raw_all['datetime'].dt.time.between(start, end)]

In [8]:
raw_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55186579 entries, 242657 to 63825230
Data columns (total 8 columns):
 #   Column              Dtype         
---  ------              -----         
 0   datetime            datetime64[ns]
 1   lng                 float64       
 2   lat                 float64       
 3   speed               float64       
 4   door_up             bool          
 5   door_down           bool          
 6   anonymized_vehicle  object        
 7   anonymized_driver   object        
dtypes: bool(2), datetime64[ns](1), float64(3), object(2)
memory usage: 3.0+ GB


In [9]:
raw_all.drop_duplicates(inplace=True)

In [10]:
raw_all = raw_all.sort_values(['anonymized_vehicle','datetime' ], ascending=[True,True])


In [11]:
raw_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54459607 entries, 244703 to 63467446
Data columns (total 8 columns):
 #   Column              Dtype         
---  ------              -----         
 0   datetime            datetime64[ns]
 1   lng                 float64       
 2   lat                 float64       
 3   speed               float64       
 4   door_up             bool          
 5   door_down           bool          
 6   anonymized_vehicle  object        
 7   anonymized_driver   object        
dtypes: bool(2), datetime64[ns](1), float64(3), object(2)
memory usage: 2.9+ GB


In [13]:
raw_all.to_csv('raw_all.csv', index=False, encoding='utf-8')

In [ ]:
rev_stop_merged = pd.read_csv('/Users/nguyentran0703/Downloads/BDC HACKATHON/Bus_route_data/raw_GPS/rev_stop_merged.csv')
stop_by_var = pd.read_csv('/Users/nguyentran0703/Downloads/BDC HACKATHON/Stop_by_vars_merged.csv')

HCM_bus_routes = pd.concat([rev_stop_merged,stop_by_var])
HCM_bus_routes.drop_duplicates('StopId',inplace=True)


In [ ]:
HCM_bus_routes[HCM_bus_routes.duplicated(keep=False)]

,StopId,Code,Name,StopType,Zone,Ward,AddressNo,Street,SupportDisability,Status,Lng,Lat,Search,Routes,Column1


In [ ]:
HCM_bus_routes.to_csv('HCM_bus_routes.csv', index=False, encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Index: 2152 entries, 0 to 1172
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   StopId             2152 non-null   int64  
 1   Code               2152 non-null   object 
 2   Name               2152 non-null   object 
 3   StopType           2152 non-null   object 
 4   Zone               2152 non-null   object 
 5   Ward               898 non-null    object 
 6   AddressNo          2152 non-null   object 
 7   Street             2152 non-null   object 
 8   SupportDisability  210 non-null    object 
 9   Status             2152 non-null   object 
 10  Lng                2152 non-null   float64
 11  Lat                2152 non-null   float64
 12  Search             2152 non-null   object 
 13  Routes             2152 non-null   object 
 14  Column1            28 non-null     object 
dtypes: float64(2), int64(1), object(12)
memory usage: 269.0+ KB


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point



df_gps = df_gps[df_gps['datetime'].dt.time.between(start, end)]
df_station = pd.read_csv("/Users/voquangkhai/Desktop/Bus_route_data/HCMC_bus_routes/1/rev_stops_by_var.csv")  # thay file thật của bạn

# --- 2. Convert sang GeoDataFrame ---
gdf_gps = gpd.GeoDataFrame(
    df_gps,
    geometry=gpd.points_from_xy(df_gps['lng'], df_gps['lat']),
    crs="EPSG:4326"
)

gdf_station = gpd.GeoDataFrame(
    df_station,
    geometry=gpd.points_from_xy(df_station['Lng'], df_station['Lat']),
    crs="EPSG:4326"
)

# --- 3. Chuyển sang hệ toạ độ mét ---
gdf_gps = gdf_gps.to_crs(epsg=3857)
gdf_station = gdf_station.to_crs(epsg=3857)

# --- 4. Join theo khoảng cách (ví dụ 30 mét) ---
distance_threshold = 10  # mét

gdf_joined = gpd.sjoin_nearest(
    gdf_gps,
    gdf_station,
    how="inner",
    max_distance=distance_threshold
)

# Kết quả: thêm cột station_id, station_name, distance_m
display(gdf_joined)